# Actividad 2:
# Comparativa de técnicas de ajuste de hiperparámetros en clasificación médica

## Objetivo
Construir un modelo de clasificación para predecir la probabilidad de diabetes en pacientes usando el conjunto de datos Pima Indians Diabetes Dataset, aplicando Grid Search y Random Search para optimizar el rendimiento del modelo y comparar sus resultados en términos de precisión, F1-score y eficiencia computacional.

**Dataset utilizado:**  
[Pima Indians Diabetes Dataset](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv)

---

### Estructura del Notebook:
1. Metodología.
2. Configuración inicial del notebook.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

---

### Flujo de trabajo
1. **Carga y exploración de datos:**
   - Estadísticas descriptivas básicas.

2. **Preprocesamiento:**
   - Detección y tratamiento de valores anómalos en base a un rango de valores biológicamente viables. Los datos fuera de este rango (outliers) distintos de cero fueron medidos en % de la muestra total, si era menor al 1% los datos se borraban, mientras que si eran mas del 1% se transformaban en cero. Para valores cero en variables en donde este valor es INCOMPATIBLE con la vida, se conviertieron en % igual en que el caso de los fuera de rango, en donde si el numero de muestras con cero era menor al 1% del total, se borraban, de otra maneta, se decidió transformar en mediana dichos valores que fueran mayores al 1%.
   - Escalado de variables numéricas (StandardScaler).
   - División estratificada del dataset (70% entrenamiento - 30% prueba).

3. **Modelado y optimización:**
   - Modelo base: Random Forest sin ajuste
   - Técnicas de optimización:
     - Grid Search (búsqueda exhaustiva en malla paramétrica)
     - Random Search (muestreo aleatorio de parámetros)
   - Métricas de evaluación: 
      - F1-Score: es la media armónica de la precisión y el recall, especialmente útil cuando hay un desequilibrio de clases en los datos.
      - Precisión: mide la exactitud de las predicciones positivas del modelo. "¿Cuantos predicciones positivas son realmente positivas?"
      - Recall: mide la capacidad del modelo para identificar todas las instancias positivas reales. "¿De todas las instancias positivas, cuantas predijo correctamente?"
      - AUC: métrica que mide la capacidad de un modelo de clasificación para distinguir entre clases. La curva ROC traza la tasa de verdaderos positivos (Recall) contra la tasa de falsos positivos (FPR) en varios umbrales de clasificación.

4. **Evaluación comparativa:**
   - Análisis de rendimiento (métricas).
   - Comparación de tiempos de ejecución.
   - Visualización de resultados.

---

# 2. Configuración inicial del notebook
- Importación de librerias necesarias.
- Configuraciones necesarias para el correcto manejo de las salidas del código.

--- 

In [ ]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configuración de pandas para mostrar todos los parámetros
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

> **Nota 2:** Los nombres de las columnas se definieron en base a la informacion disponible en `kaggle` sobre este data set: 
[Pima Indians Diabetes Dataset (Kaggle)](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database), ya que en el link de github no se encontraban.

---

**Bloque 1:** Funciones de preprocesamiento de datos.

- **`load_data()`** 
Carga el dataset, define las columnas y lo convierte en un DataFrame con **pandas**. Luego hace una exploración inicial del df mostrando estadísticas básicas, además de la distribución de personas con y sin diabetes.

- **`preprocess_data()`** 
Preprocesa el dataframe tratando outliers, escalando los datos y finalmente dividiendo el dataset en datos de entrenamiento y de prueba.

In [ ]:
def load_and_explore_data(url: str) -> tuple[pd.DataFrame, pd.DataFrame, pd.Series]:
    """
    Carga un dataset de diabetes desde una URL, asigna nombres de columnas y entrega estadísticas exploratorias.

    Este proceso incluye:
    - Lectura del archivo CSV desde la URL proporcionada.
    - Asignación de nombres a las columnas según el dataset de diabetes de PIMA.
    - Cálculo de estadísticas básicas (media, mediana, moda, percentiles, mínimo y máximo).
    - Cálculo de la distribución porcentual de la variable objetivo `Outcome`.

    Args:
        url (str): Ruta o URL del archivo CSV que contiene el dataset.

    Returns:
        tuple:
            - df (pd.DataFrame): DataFrame original con los datos cargados.
            - stats_df (pd.DataFrame): Estadísticas descriptivas por variable numérica.
            - outcome_dist (pd.Series): Distribución porcentual del Outcome (0 = no diabético, 1 = diabético).
    """
    # Asignación de nombres de columna
    column_names = [
        "Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
        "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"
    ]
    df = pd.read_csv(url, names=column_names)

    # Estadísticas básicas
    stats_df = pd.DataFrame({
        "mean": df.mean(),
        "median": df.median(),
        "mode": df.mode().iloc[0],
        "25%": df.quantile(0.25),
        "75%": df.quantile(0.75),
        "min": df.min(),
        "max": df.max()
    })

    # Distribución porcentual de Outcome
    outcome_dist = df["Outcome"].value_counts(normalize=True) * 100
    outcome_dist = outcome_dist.rename("percentage (%)")

    return df, stats_df, outcome_dist

def prepare_data(
    df: pd.DataFrame,
    target_col: str = "Outcome",
    test_size: float = 0.3,
    random_state: int = 42,
    verbose: bool = True
) -> tuple:
    """
    Prepara el dataset para entrenamiento aplicando limpieza, imputación, escalado y división.

    El proceso completo incluye:
    1. Detección y tratamiento de valores fuera de rango (outliers) distintos de cero:
        - Reemplazo con cero si superan el 1% de la columna.
        - Eliminación de filas si representan menos del 1%.
    2. Imputación de ceros usando la mediana (si hay más del 1%) o eliminación de filas.
    3. Escalado de variables numéricas con StandardScaler.
    4. División estratificada del dataset en conjunto de entrenamiento y prueba (train/test).

    Args:
        df (pd.DataFrame): DataFrame original con los datos.
        target_col (str, opcional): Nombre de la columna objetivo. Por defecto es "Outcome".
        test_size (float, opcional): Proporción del dataset para el conjunto de prueba. Default = 0.3.
        random_state (int, opcional): Semilla aleatoria para reproducibilidad. Default = 42.
        verbose (bool, opcional): Si True, imprime detalles del proceso. Default = True.

    Returns:
        tuple:
            - X_train (np.ndarray): Conjunto de entrenamiento (features escaladas).
            - X_test (np.ndarray): Conjunto de prueba (features escaladas).
            - y_train (pd.Series): Etiquetas de entrenamiento.
            - y_test (pd.Series): Etiquetas de prueba.
            - scaler (StandardScaler): Objeto escalador ajustado.
    """
    # 1. Limpieza
    valid_ranges = {
        "Glucose": (40, 500),
        "BloodPressure": (40, 180),
        "SkinThickness": (10, 80),
        "Insulin": (10, 900),
        "BMI": (10, 70),
        "DiabetesPedigreeFunction": (0.05, 2.5),
        "Age": (10, 120)
    }

    df_clean = df.copy()
    original_rows = df.shape[0]
    dropped_1, zeroed_1 = 0, 0

    # 1. Tratamiento de outliers distintos de cero
    if verbose:
        print("=== Limpieza de valores fuera de rango (excluyendo ceros) ===")
    
    for col, (min_val, max_val) in valid_ranges.items():
        # Detectar valores fuera de rango que NO sean cero
        non_zero_mask = df_clean[col] != 0
        outliers = non_zero_mask & ((df_clean[col] < min_val) | (df_clean[col] > max_val))
        
        pct_out = outliers.mean() * 100
        count_out = outliers.sum()
        
        if verbose:
            print(f"{col}: {pct_out:.2f}% ({count_out}) valores no-cero fuera de rango")
        
        if pct_out > 1:
            # Reemplazar SOLO valores fuera de rango (no-cero) con 0
            df_clean.loc[outliers, col] = 0
            zeroed_1 += count_out
        else:
            # Eliminar filas con valores fuera de rango (no-cero)
            df_clean = df_clean[~outliers]
            dropped_1 += count_out

    # 2. Imputación de ceros
    dropped_2, imputed_2 = 0, 0
    if verbose:
        print("\n=== Tratamiento de ceros ===")
    
    # Convertir columnas numéricas a float antes de la imputación
    float_cols = ["BMI", "DiabetesPedigreeFunction", "Insulin"]
    for col in float_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(float)
    
    for col in valid_ranges:
        zeros = df_clean[col] == 0
        pct_zeros = zeros.mean() * 100
        count_zeros = zeros.sum()
        
        if verbose:
            print(f"{col}: {pct_zeros:.2f}% ceros ({count_zeros})")
        
        if pct_zeros > 1:
            # Calcular mediana y convertir al tipo de dato original
            median = df_clean.loc[df_clean[col] != 0, col].median()
            
            # Preservar tipo de dato
            if df_clean[col].dtype == int:
                median = round(median)
            
            df_clean.loc[zeros, col] = median
            imputed_2 += count_zeros
        else:
            df_clean = df_clean.loc[~zeros]
            dropped_2 += count_zeros

    if verbose:
        print("\n=== Resumen limpieza ===")
        print(f"Filas originales: {original_rows}")
        print(f"Filas tras limpieza: {df_clean.shape[0]}")
        print(f"Eliminados por outliers: {dropped_1}")
        print(f"Seteados a cero por outliers: {zeroed_1}")
        print(f"Eliminados por ceros: {dropped_2}")
        print(f"Imputados con mediana: {imputed_2}")

    # 3. Preprocesamiento
    X = df_clean.drop(columns=[target_col])
    y = df_clean[target_col]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, random_state=random_state, stratify=y
    )

    return X_train, X_test, y_train, y_test, scaler

**Bloque 2:** Funciones de modelado.

- **`train_base_model()`** 
Entrena un modelo base con parametros por defecto para futuras comparaciones de metricas, incluyendo el tiempo.

- **`evaluate_model()`** 
Evalúa el rendimiento del modelo dado sobre el conjunto de datos de prueba, incluyendo las métricas a evaluar como F1-Score, precisión, recall y AUC.

In [ ]:
def train_base_model(X_train, y_train):
    """
    Entrena un modelo base Random Forest con parámetros por defecto.

    El modelo se entrena con:
    - Pesos balanceados para clases desbalanceadas (`class_weight='balanced'`).
    - Uso de todos los núcleos disponibles (`n_jobs=-1`).
    - Semilla fija para reproducibilidad (`random_state=42`).

    Args:
        X_train (np.ndarray o pd.DataFrame): Características del conjunto de entrenamiento.
        y_train (pd.Series o np.ndarray): Etiquetas del conjunto de entrenamiento.

    Returns:
        tuple:
            - model (RandomForestClassifier): Modelo entrenado.
            - duration (float): Tiempo de entrenamiento en segundos.
    """
    start = time.time()
    model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
    model.fit(X_train, y_train)
    return model, time.time() - start

def evaluate_model(model, X_test, y_test):
    """
    Evalúa un modelo entrenado en el conjunto de prueba utilizando varias métricas de clasificación.

    Las métricas calculadas son:
    - F1-Score
    - Precisión
    - Recall
    - AUC (Area Under the ROC Curve)

    Args:
        model (RandomForestClassifier u otro clasificador con predict_proba): Modelo entrenado.
        X_test (np.ndarray o pd.DataFrame): Características del conjunto de prueba.
        y_test (pd.Series o np.ndarray): Etiquetas verdaderas del conjunto de prueba.

    Returns:
        dict: Diccionario con las métricas:
            - "F1-Score": Valor del F1-score.
            - "Precisión": Valor de la precisión.
            - "Recall": Valor del recall.
            - "AUC": Área bajo la curva ROC.
    """
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    return {
        "F1-Score": f1_score(y_test, y_pred),
        "Precisión": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_proba)
    }

**Bloque 3:** Funciones de optimización.

- **`optimize_with_gridsearch()`** 
Optimización utilizando **GridSearchCV**.

- **`optimize_with_randomsearch()`** 
Optimización utilizando **RandomizedSearchCV**.

In [ ]:
def optimize_with_gridsearch(X_train, y_train, X_test, y_test):
    """
    Optimiza un modelo Random Forest utilizando Grid Search con validación cruzada.

    El proceso evalúa combinaciones exhaustivas de hiperparámetros definidos 
    en un grid, utilizando AUC como métrica de evaluación principal.

    Args:
        X_train (np.ndarray o pd.DataFrame): Datos de entrenamiento (features).
        y_train (pd.Series o np.ndarray): Etiquetas del conjunto de entrenamiento.
        X_test (np.ndarray o pd.DataFrame): Datos de prueba (features).
        y_test (pd.Series o np.ndarray): Etiquetas del conjunto de prueba.

    Returns:
        tuple:
            - best_model (RandomForestClassifier): Modelo entrenado con los mejores hiperparámetros.
            - metrics (dict): Métricas del modelo sobre el conjunto de prueba, incluyendo:
                - "accuracy": Precisión del modelo.
                - "recall": Recall.
                - "f1": F1-score.
                - "auc": Área bajo la curva ROC.
                - "best_params": Diccionario de hiperparámetros óptimos.
                - "train_time": Tiempo total de entrenamiento en segundos.
    """
    param_grid = {
        "n_estimators": [50, 100, 150],
        "max_depth": [5, 10, 15],
        "min_samples_split": [2, 5, 10]
    }

    start = time.time()
    grid_search = GridSearchCV(
        RandomForestClassifier(random_state=42),
        param_grid=param_grid,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)
    end = time.time()

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
        "auc": roc_auc_score(y_test, y_proba),
        "best_params": grid_search.best_params_,
        "train_time": end - start
    }

    return best_model, metrics

def optimize_with_randomsearch(X_train, y_train, X_test, y_test):
    """
    Optimiza un modelo Random Forest utilizando Randomized Search con validación cruzada.

    A diferencia del Grid Search, este método explora aleatoriamente combinaciones
    de hiperparámetros en un espacio definido, lo cual puede reducir el tiempo de cómputo.

    Args:
        X_train (np.ndarray o pd.DataFrame): Datos de entrenamiento (features).
        y_train (pd.Series o np.ndarray): Etiquetas del conjunto de entrenamiento.
        X_test (np.ndarray o pd.DataFrame): Datos de prueba (features).
        y_test (pd.Series o np.ndarray): Etiquetas del conjunto de prueba.

    Returns:
        tuple:
            - best_model (RandomForestClassifier): Modelo entrenado con los mejores hiperparámetros.
            - metrics (dict): Métricas del modelo sobre el conjunto de prueba, incluyendo:
                - "accuracy": Precisión del modelo.
                - "recall": Recall.
                - "f1": F1-score.
                - "auc": Área bajo la curva ROC.
                - "best_params": Diccionario de hiperparámetros óptimos.
                - "train_time": Tiempo total de entrenamiento en segundos.
    """
    param_dist = {
        "n_estimators": randint(50, 200),
        "max_depth": randint(5, 30),
        "min_samples_split": randint(2, 20)
    }

    start = time.time()
    random_search = RandomizedSearchCV(
        RandomForestClassifier(random_state=42),
        param_distributions=param_dist,
        n_iter=100,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        random_state=42
    )
    random_search.fit(X_train, y_train)
    end = time.time()

    best_model = random_search.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
        "auc": roc_auc_score(y_test, y_proba),
        "best_params": random_search.best_params_,
        "train_time": end - start
    }

    return best_model, metrics

**Bloque 4:** Funciones de visualización.

- **`create_comparison_table()`** 
Crea un DataFrame con la comparación de las métricas del modelo base y las dos optimizaciones hechas.

- **`visualize_results()`** 
Crea dos gráficos que comparan el rendimiento y los tiempos respectivamente.

In [ ]:
def create_comparison_table(results_dict):
    """
    Crea una tabla comparativa con métricas de desempeño para distintos modelos.

    La tabla incluye precisión, recall, F1-score, AUC, tiempo de entrenamiento y 
    los mejores hiperparámetros encontrados (si aplica).

    Args:
        results_dict (dict): Diccionario con resultados de modelos. Cada entrada debe tener
            el nombre del modelo como clave y un subdiccionario con claves "metrics"
            (que incluya F1-Score, accuracy, recall, AUC, etc.) y "train_time".

    Returns:
        pd.DataFrame: DataFrame ordenado por F1-Score descendente, con columnas:
            - Model
            - Accuracy
            - Recall
            - F1-Score
            - AUC
            - Train Time (s)
            - Best Params
    """
    rows = []
    for model_name, info in results_dict.items():
        metrics = info.get("metrics", {})
        train_time = metrics.get("train_time", None)
        best_params = metrics.get("best_params", None)

        row = {
            "Model": model_name,
            "Accuracy": metrics.get("accuracy", metrics.get("Precisión", None)),
            "Recall": metrics.get("recall", metrics.get("Recall", None)),
            "F1-Score": metrics.get("f1", metrics.get("F1-Score", None)),
            "AUC": metrics.get("auc", metrics.get("AUC", None)),
            "Train Time (s)": train_time,
            "Best Params": str(best_params) if best_params else "-"
        }
        rows.append(row)

    df_comparison = pd.DataFrame(rows)
    return df_comparison.sort_values(by="F1-Score", ascending=False).reset_index(drop=True)

def visualize_results(df_comparison):
    """
    Genera visualizaciones comparativas de desempeño entre modelos.

    Crea dos gráficos:
    - Gráfico de barras agrupadas para comparar métricas de clasificación (Accuracy, Recall, F1-Score, AUC).
    - Gráfico de barras para comparar los tiempos de entrenamiento.

    Args:
        df_comparison (pd.DataFrame): DataFrame generado por `create_comparison_table()`, que
            contiene métricas y tiempos de modelos entrenados.
    """
    # Configuración de estilo
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(16, 7))
    
    # --- GRÁFICO 1: COMPARACIÓN DE MÉTRICAS ---
    plt.subplot(1, 2, 1)
    
    # Preparamos los datos en formato largo
    metrics = ["Accuracy", "Recall", "F1-Score", "AUC"]
    df_melted = df_comparison.melt(id_vars="Model", 
                                  value_vars=metrics,
                                  var_name="Metric", 
                                  value_name="Value")
    
    # Creamos el gráfico de barras agrupadas
    palette = sns.color_palette("Set2", len(df_comparison))
    ax = sns.barplot(x="Metric", 
                     y="Value", 
                     hue="Model", 
                     data=df_melted, 
                     palette=palette,
                     width=0.8)  
    
    # Añadimos etiquetas de valores
    for p in ax.patches:
        ax.annotate(f"{p.get_height():.3f}", 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', 
                    xytext=(0, 9), 
                    textcoords='offset points',
                    fontsize=9)
    
    plt.title("Comparación de Métricas por Modelo", fontsize=16)
    plt.xlabel("Métricas", fontsize=12)
    plt.ylabel("Valor", fontsize=12)
    plt.ylim(0, 1)
    
    # Mover leyenda al centro superior
    plt.legend(title="Modelo", loc='upper center', ncol=3, frameon=True)
    
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # --- GRÁFICO 2: TIEMPOS DE ENTRENAMIENTO ---
    plt.subplot(1, 2, 2)
    
    # Ordenamos por tiempo para mejor visualización
    df_time = df_comparison.sort_values("Train Time (s)", ascending=False)
    
    # Creamos el gráfico de barras para tiempos con la misma paleta
    ax2 = sns.barplot(x="Model", 
                      y="Train Time (s)", 
                      data=df_time, 
                      hue="Model",  # Corregimos la advertencia
                      palette=palette,
                      width=0.6,
                      legend=False)
    
    # Añadimos etiquetas de tiempo
    for p in ax2.patches:
        ax2.annotate(f"{p.get_height():.2f}s", 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', 
                     xytext=(0, 9), 
                     textcoords='offset points',
                     fontsize=9) # Ocultamos leyenda redundante
    
    plt.title("Tiempos de Entrenamiento", fontsize=16)
    plt.xlabel("Modelos", fontsize=12)
    plt.ylabel("Segundos", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Ajustar espacio entre subplots
    plt.tight_layout(pad=3.0)
    
    plt.savefig("comparacion_de_modelos.png")

    # Mostrar gráficos
    plt.show()

**Bloque 5:** Función de ejecución.

- **`main()`**
Utiliza todas las funciones definidas anteriormente para obtener datos del modelo base y de los modelos optimizados con **GridSearchCV**, **RandomizedSearchCV** y **Optuna**.  

In [ ]:
def main():
    """
    Función principal que orquesta todo el flujo de trabajo de un proyecto de clasificación
    de diabetes tipo 2 utilizando Random Forest y diferentes métodos de optimización de hiperparámetros.

    El pipeline incluye:
        1. Carga y exploración de datos desde una URL.
        2. Limpieza de datos y preprocesamiento (tratamiento de outliers y ceros).
        3. Entrenamiento de un modelo base con RandomForestClassifier.
        4. Optimización de hiperparámetros usando:
            - Grid Search
            - Random Search
        5. Evaluación de los modelos usando múltiples métricas (Accuracy, Recall, F1-Score, AUC).
        6. Comparación tabular y análisis de los resultados.
        7. Visualización de métricas y tiempos de entrenamiento.

    Esta función imprime resultados intermedios y finales en consola, incluyendo estadísticas
    descriptivas, métricas, parámetros óptimos y análisis comparativo.
    """
    # Configuración inicial
    URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
    RANDOM_STATE = 42
    
    # 1. Carga de datos
    print("\nCargando y explorando datos...")
    df, stats_df, outcome_dist = load_and_explore_data(URL)
    
    print("\nDistribución de Outcome (%):\n")
    print(outcome_dist.to_string())
    print("\nEstadísticas descriptivas:\n")
    print(stats_df.to_string())
    
    # 2. Preparación de datos
    print("\nPreparando datos...\n")
    X_train, X_test, y_train, y_test, scaler = prepare_data(df, verbose=True)
    
    # 3. Modelo base
    print("\nEntrenando modelo base...")
    base_model, base_time = train_base_model(X_train, y_train)
    base_metrics = evaluate_model(base_model, X_test, y_test)
    print("\nMétricas del modelo base:\n")
    for metric, value in base_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    # 4. Optimización con GridSearch
    print("\nOptimizando con GridSearch...")
    grid_model, grid_metrics = optimize_with_gridsearch(X_train, y_train, X_test, y_test)
    
    # 5. Optimización con RandomSearch
    print("\nOptimizando con RandomSearch...")
    random_model, random_metrics = optimize_with_randomsearch(X_train, y_train, X_test, y_test)
    
    # 6. Comparación de resultados
    results_dict = {
        "Base Model": {
            "model": base_model,
            "metrics": {
                **base_metrics,
                "accuracy": accuracy_score(y_test, base_model.predict(X_test)),
                "train_time": base_time
            }
        },
        "GridSearch": {
            "model": grid_model,
            "metrics": grid_metrics
        },
        "RandomSearch": {
            "model": random_model,
            "metrics": random_metrics
        }
    }
    
    # Crear tabla comparativa
    df_comparison = create_comparison_table(results_dict)
    
    print("\n" + "="*70)
    print("RESULTADOS FINALES")
    print("="*70)
    
    # Formato mejorado para la tabla
    print("\n" + "-"*70)
    print("COMPARACIÓN DE MODELOS")
    print("-"*70)
    print(df_comparison.drop(columns="Best Params").to_string(index=False))
    
    print("\n" + "-"*70)
    print("PARÁMETROS ÓPTIMOS")
    print("-"*70)
    for i, row in df_comparison.iterrows():
        print(f"\n{row['Model']}:")
        if row['Best Params'] != '-':
            params = eval(row['Best Params'])
            for key, value in params.items():
                print(f"  {key}: {value}")
        else:
            print("  Parámetros por defecto")
    
    # Análisis de resultados
    print("\n" + "-"*70)
    print("ANÁLISIS DE RESULTADOS")
    print("-"*70)
    
    base_accuracy = df_comparison[df_comparison['Model'] == 'Base Model']['Accuracy'].values[0]
    grid_accuracy = df_comparison[df_comparison['Model'] == 'GridSearch']['Accuracy'].values[0]
    random_accuracy = df_comparison[df_comparison['Model'] == 'RandomSearch']['Accuracy'].values[0]

    base_recall = df_comparison[df_comparison['Model'] == 'Base Model']['Recall'].values[0]
    grid_recall = df_comparison[df_comparison['Model'] == 'GridSearch']['Recall'].values[0]
    random_recall = df_comparison[df_comparison['Model'] == 'RandomSearch']['Recall'].values[0]
    
    base_f1 = df_comparison[df_comparison['Model'] == 'Base Model']['F1-Score'].values[0]
    grid_f1 = df_comparison[df_comparison['Model'] == 'GridSearch']['F1-Score'].values[0]
    random_f1 = df_comparison[df_comparison['Model'] == 'RandomSearch']['F1-Score'].values[0]

    print(f"\n1. Mejora en Accuracy:")
    print(f"   - GridSearch: +{(grid_accuracy - base_accuracy)*100:.1f}% vs Base Model")
    print(f"   - RandomSearch: +{(random_accuracy - base_accuracy)*100:.1f}% vs Base Model")
    
    print(f"\n1. Mejora en Recall (detección de positivos):")
    print(f"   - GridSearch: +{(grid_recall - base_recall)*100:.1f}% vs Base Model")
    print(f"   - RandomSearch: +{(random_recall - base_recall)*100:.1f}% vs Base Model")
    
    print(f"\n2. Mejora en F1-Score (equilibrio precisión-recall):")
    print(f"   - GridSearch: +{(grid_f1 - base_f1)*100:.1f}% vs Base Model")
    print(f"   - RandomSearch: +{(random_f1 - base_f1)*100:.1f}% vs Base Model")
    
    print(f"\n3. Tiempo de entrenamiento:")
    print(f"   - GridSearch fue {df_comparison['Train Time (s)'].iloc[0]/df_comparison['Train Time (s)'].iloc[2]:.0f}x más lento que Base Model")
    print(f"   - RandomSearch fue {df_comparison['Train Time (s)'].iloc[1]/df_comparison['Train Time (s)'].iloc[2]:.0f}x más lento que Base Model")

    # Visualización
    print("\nGenerando visualizaciones...")
    visualize_results(df_comparison)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Reflexión Final

> **Nota:** Debido a que hay variaciones entre ejecuciones, se puede esperar que algunos datos como la comparación de eficiencia en tiempo tenga datos levemente diferentes de los que se muestren al momento de ejecutar nuevamente el notebook, por lo que se deben tomar mas como **aproximaciones** y no como un resultado exacto.

## 1. ¿Cuál técnica fue más eficiente?
- Random Search demostró ser más eficiente:
    - Velocidad: 7s vs 16s de Grid Search.
    - Recursos computacionales: Menor consumo de CPU/memoria.
    - Costo-beneficio: Mejor equilibrio entre tiempo y mejora de métricas.
    - Escalabilidad: Más adecuado para espacios de búsqueda grandes.

¿Por qué sucede esto?
- Random Search explora eficientemente espacios hiperparamétricos amplios con menos iteraciones, mientras que Grid Search sufre del "curse of dimensionality" - el número de combinaciones crece exponencialmente con cada nuevo parámetro añadido.

## 2. ¿Cuál encontró el mejor modelo?
- Basados en F1-Score, Random Search fue mejor modelo, principalmente en términos de:
    - Acurracy: 2.6% de mejora en relación al modelo base, y 0.4% mejor que Grid Search.
    - F1-Score: 2.6% mejor que el modelo base y 1.5% mejor que el Grid Search.
    - Balance general: Mejor equilibrio entre precisión y exhaustividad
- Recall fue igual al del modelo base, y mayor al de Grid Search, AUC fue levemente mejor en Grid Search.

## 3. ¿Qué hubieras hecho diferente?
Mejoras clave en el flujo de trabajo:
- Probar diferentes numeros de iteraciones, ya que Random Search hace busquedas aleatorias podríamos probar espacios de busqueda mas amplios.
- Feature Engineering avanzado:
- Manejo de desbalanceo con SMOTE:
- Optimización bayesiana:
- Validación cruzada estratificada:
- Ensamblaje de modelos:

Priorización de métricas:

- Para problemas médicos como detección de diabetes, hubiera enfocado más en:
    - Maximizar Recall (minimizar falsos negativos)
    - Optimizar curva Precision-Recall en lugar de solo AUC
    - Usar métricas específicas como Sensibilidad y Especificidad

- Análisis de características:
    - Realizar análisis de importancia con SHAP